BUG RELATED QUESTIONS
- Why do we need to add a model with parser.AddModelFromUrl if we have already added a model? Otherwise, we run into trouble later (can show what happens)
- Why does the simulation stop when I add point clouds (this is when we find the antipodal grasp)?
- How to deal with context: keep one context all the time? What is the best way of dealing with this?
- Station input port seeming to have 20 variables as opposed to 12 (https://piazza.com/class/lm1wm4iwue34wl/post/526)
- Simulation time being slow and also rendering issues with point cloud being the only thing that's updating when we try to simulate our trajectory

EXPERIMENTAL CODE QUESTIONS
- Planner: how to design the planner? Is that design the right approach?
- Perception: more than one book? Get ICP for more than one book? How do we segment the point cloud?
- IK/DiffIK: how do we fix the z-coord in the base?

## Generating and Visualizing Antipodal Grasps on Book

In [1]:
import numpy as np
import trimesh
import sys
from typing import List 
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RigidTransform,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    Context,
    Diagram,
    PointCloud,
    Simulator,
    TrajectorySource,
    Solve,
    RotationMatrix,
    MultibodyPlant
)
from pydrake.geometry import Meshcat
from pydrake.multibody import inverse_kinematics

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb, AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import ConfigureParser
from manipulation.clutter import GraspCandidateCost, GenerateAntipodalGraspCandidate
from manipulation.icp import IterativeClosestPoint

# Own utils
from hwstation.utils import setup_builder, plot_and_simulate
from hwstation.add_objects import get_library_scenario_data, get_library_scenario_data_without_robot

from enum import Enum


Starting Meshcat

In [2]:
# Start meshcat
try:
    meshcat = Meshcat(7005)
except:
    pass #This error appears if this cell is executed twice (port 7006 is already taken then)

INFO:drake:Meshcat listening for connections at http://localhost:7005


## 1. Setup environment

In [3]:
scenario_data = get_library_scenario_data()
print(scenario_data)


cameras:
    camera0:
        name: table_camera_0
        depth: True
        X_PB:
            base_frame: camera_table_above
    
    camera1:
        name: table_camera_1
        depth: True
        X_PB:
            base_frame: camera_table_left
    
    camera2:
        name: table_camera_2
        depth: True
        X_PB:
            base_frame: camera_table_right

    camera3:
        name: table_camera_3
        depth: True
        X_PB:
            base_frame: camera_behind_table
directives:
- add_model:
    name: ground_floor
    file: file:///workspaces/RobotLibrarian/hwstation/objects/ground_floor.sdf
- add_weld:
    parent: world
    child: ground_floor::ground_floor
    X_PC:
        translation: [0.0,0.0,0.0]

- add_model:
    name: shelf_A
    file: file:///workspaces/RobotLibrarian/hwstation/objects/library_shelf.sdf
- add_weld:
    parent: world
    child: shelf_A::shelves_body
    X_PC:
        translation: [1.7, 2.4, 0.42]

- add_model:
    name: shelf_B
    file

In [21]:
 scenario_data = get_library_scenario_data()

In [17]:
def init_lib_environment():
    # Get string describing scenario
    scenario_data = get_library_scenario_data()

    #Setting up all drake simulation objects:
    builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)

    # Allowing the Parser to access all of the aspects in the environment including the robot (iiwa)
    parser.AddModelsFromUrl("file:///workspaces/RobotLibrarian/hwstation/objects/mobile_iiwa.dmd.yaml")

    #Simulate environment (right now, only the books fall on the table)
    diagram, simulator, sim_context = plot_and_simulate(meshcat, builder, plant, station, time_end=1.0)

    return diagram, plant, builder, station, simulator, scene_graph, sim_context


In [18]:
diagram, plant, builder, station, simulator, scene_graph, sim_context = init_lib_environment()
plant.Finalize()
diagram_context = diagram.CreateDefaultContext()

In [20]:
plant.GetBodyByName("ASD")

RuntimeError: GetBodyByName(): There is no Body named 'ASD' anywhere in the model (valid names are: base_x, base_y, body, iiwa_link_0, iiwa_link_1, iiwa_link_2, iiwa_link_3, iiwa_link_4, iiwa_link_5, iiwa_link_6, iiwa_link_7, iiwa_link_ee, iiwa_link_ee_kuka, left_finger, right_finger, world)

In [6]:
# from hwstation.utils import visualize_diagram
# visualize_diagram(diagram)

## 2. ICP for book point cloud

Pointcloud Related Functions (to be used when finding point cloud to do grasp)

In [7]:
def get_table_pointclouds(diagram_context: Context, diagram: Diagram):
    point_cloud_dict = {}
    for idx in range(4):
        point_cloud_dict[f"table_camera_{idx}_ptcloud"] = diagram.GetOutputPort(f"table_camera_{idx}_ptcloud").Eval(diagram_context)
    return point_cloud_dict


def merge_point_clouds(table_pointclouds: dict, 
                        downsample_factor: float = 0.005,
                        lower_xyz: List[float] = [0.0, -0.2, 0.5564], 
                        upper_xyz: List[float] = [0.75, 1.5, 0.8]):
    pcd = []
    for key in table_pointclouds.keys():
        cloud = table_pointclouds[key]
        pcd.append(
            cloud.Crop(lower_xyz=lower_xyz, upper_xyz=upper_xyz)
            )
    merged_pcd = Concatenate(pcd)
    down_sampled_pcd = merged_pcd.VoxelizedDownSample(voxel_size=0.005)
    return down_sampled_pcd

def get_merged_pointcloud(diagram_context: Context, diagram: Diagram):
    
    #Get merged point cloud from all cameras:
    table_pointclouds = get_table_pointclouds(diagram_context, diagram)
    merged_pcd = merge_point_clouds(table_pointclouds)

    #Ensure that all number are finite:
    merged_pcd_np = merged_pcd.xyzs().transpose()
    mask_points = (merged_pcd_np== np.inf).all(axis=1)
    if mask_points.any():
        sys.exit("No infinite points were expected")
    return merged_pcd

def convert_obj_to_pc(filename: str, n_samples: int = 10000, show: bool =False) -> np.ndarray:
    book_mesh = trimesh.load(filename)
    book_hull = book_mesh.convex_hull
    sample_points = book_hull.sample(n_samples)
    point_cloud = trimesh.points.PointCloud(sample_points)
    if show:
        scene = trimesh.Scene([book_hull, point_cloud])
        scene.show()
    return np.array(point_cloud.vertices).transpose()

## 3. Finding and Visualizing the Gripper Grasp Poses

In [8]:
# Another diagram for the objects the robot "knows about": shelves, table, cameras, books (only one for now), and a gripper.  Think of this as the model in the robot's head.
def make_internal_model():
    # Get string describing scenario
    scenario_data = get_library_scenario_data_without_robot()

    #Setting up all drake simulation objects:
    builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)

    # This parser has all items in the environment except the robot and instead has a "floating" gripper to try and find antipodal grasps with
    parser.AddModelsFromUrl("file:///workspaces/RobotLibrarian/hwstation/objects/library_setup_floating_gripper.dmd.yaml")
    plant.Finalize()
    return builder.Build()

In [9]:
# The main process for finding the grasp poses and visualizing them
def sample_grasps_example(environment, environment_context, environment_plant):
    
    rng = np.random.default_rng()

    # Point clouds to obtain the cloud we will work with for grasps:
    scene_pcl = get_merged_pointcloud(environment_context, environment)
    meshcat.SetObject("merged_cropped_pcl", cloud=scene_pcl, point_size=0.004)

    book_filename = "hwstation/objects/book.obj"
    model_pcl = convert_obj_to_pc(book_filename, show=False)

    height_table = 0.5
    initial_guess = RigidTransform(p=[0.0,0.0,height_table])

    X_MS_hat, chat = IterativeClosestPoint(
        p_Om=model_pcl,
        p_Ws=scene_pcl.xyzs(),
        X_Ohat=initial_guess,
        meshcat=meshcat,
        meshcat_scene_path="icp",
        max_iterations=25,
    )

    transformed_model_pcl = X_MS_hat @ model_pcl

    cloud = PointCloud(transformed_model_pcl.shape[1])
    cloud.mutable_xyzs()[:] = transformed_model_pcl
    cloud.EstimateNormals(radius=0.5, num_closest=50)

    # Now find grasp poses
    # X_Gs will have the poses to be used for planning when working on that step
    internal_model = make_internal_model()
    internal_model_context = internal_model.CreateDefaultContext()
    costs = []
    X_Gs = []
    for i in range(100 if running_as_notebook else 2):
        cost, X_G = GenerateAntipodalGraspCandidate(
            internal_model, internal_model_context, cloud, rng
        )
        if np.isfinite(cost):
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:5]
    min_cost_XGs = []
    for idx in indices:
        min_cost_XGs.append(X_Gs[idx])

    #Get the antipodal grasp that is closest to the robot WSG frame:
    positions = np.stack([frame.translation() for frame in min_cost_XGs])

    plant_context = environment_plant.CreateDefaultContext()

    wsg = environment_plant.GetBodyByName("body")
    wsg_body_index = wsg.index()

    wsg_pose = environment_plant.EvalBodyPoseInWorld(plant_context,wsg)
    wsg_pos = wsg_pose.translation()

    best_grasp_idx = np.argmin(((positions-wsg_pos)**2).sum(axis=1))
    X_G_optim = min_cost_XGs[best_grasp_idx]

    return X_G_optim

In [10]:
optim_grasp = sample_grasps_example(diagram, diagram_context, plant)
optim_grasp

fixing improper rotation


RuntimeError: GetBodyByName(): There is no Body named 'body' anywhere in the model (valid names are: world)

In [10]:
from manipulation.pick import *

In [11]:
def get_table_wsg_poses(diagram_context: Context, plant: MultibodyPlant):
    wsg = plant.GetBodyByName("body")
    wsg_body_index = wsg.index()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    wsg_initial_pose = plant.EvalBodyPoseInWorld(plant_context,wsg)
    X_G_dict = {"initial":  wsg_initial_pose, "pick": optim_grasp, "place": wsg_initial_pose}
    return X_G_dict

def create_gripper_pose_and_command_trajectories(X_G_dict):
    traj_frames, traj_times = MakeGripperFrames(X_G_dict)
    trajectory = MakeGripperPoseTrajectory(traj_frames,traj_times)
    traj_wsg_command = MakeGripperCommandTrajectory(traj_times)
    return traj_times, trajectory, traj_wsg_command

def create_t_and_pose_list(traj_times, trajectory) -> List[RigidTransform]:
    # max_time = max(traj_times.values())
    max_time = 1.0
    t_lst = np.linspace(0, max_time, 30)
    pose_lst = []
    for t in t_lst:
        #AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=0.2)
        pose_lst.append(RigidTransform(trajectory.value(t)))
    return t_lst, pose_lst

def create_q_knots(pose_lst: List[RigidTransform], t_lst: np.ndarray, plant: MultibodyPlant, plant_context: Context):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 12-dimensional because the first 3 dimensions are the base
    and the last 2 contain gripper joints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")

    #Get base transformation
    base_y_body = plant.GetBodyByName("base_y")
    base_y_body_index = base_y_body.index()
    base_y_pose = plant.EvalBodyPoseInWorld(plant_context,base_y_body)
    base_trans = base_y_pose.translation()

    q_nominal = np.array(
        [base_trans[0], base_trans[1], base_trans[2], 0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0]
    )  # nominal joint angles for joint-centering.

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )

    for i in range(len(pose_lst)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        X_WG = pose_lst[i]
        p_WG = X_WG.translation()
        R_WG = X_WG.rotation()
        d = 0.05
        p_WG_adjustment = 0.1*np.array([1.0,1.0,1.0])

        AddPositionConstraint(ik, p_WG - p_WG_adjustment, p_WG + p_WG_adjustment)
        AddOrientationConstraint(ik, R_WG, 0.05)
        
        prog.AddQuadraticErrorCost(0.1*np.identity(len(q_variables)), q_nominal, q_variables)
        
        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
        else:
            prog.SetInitialGuess(q_variables, q_knots[i-1][:12])
        
        ################################################

        result = Solve(prog)

        assert result.is_success()

        q_vars = result.GetSolution(q_variables)
        # print(q_vars)
        q_vars = np.hstack((q_vars, np.zeros(8)))
        q_knots.append(q_vars)
    
    q_knots = np.array(q_knots)

    return q_knots

In [12]:
def create_book_picking_joint_trajectory(diagram: Diagram, plant: MultibodyPlant, diagram_context: Context, optim_grasp: RigidTransform):
    X_G_dict = get_table_wsg_poses(diagram_context, plant)
    traj_times, trajectory, traj_wsg_command = create_gripper_pose_and_command_trajectories(X_G_dict)
    t_lst, pose_lst = create_t_and_pose_list(traj_times, trajectory)
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    q_knots = create_q_knots(pose_lst, t_lst, plant, plant_context)
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots.T)
    return q_traj, t_lst, q_knots, traj_wsg_command

def plot_joint_commands(q_knots):
    import pandas as pd
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(9,9))
    pd.DataFrame(q_knots,columns=plant.GetActuatorNames()).plot(ax=ax)

In [13]:
q_traj, t_lst, q_knots, traj_wsg_command = create_book_picking_joint_trajectory(diagram, plant, diagram_context, optim_grasp)
# plot_joint_commands(q_knots)

In [14]:
def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    # AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )
    # visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)


    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("mobile_iiwa.desired_state")
    )

    diagram = builder.Build()
    plant.Finalize()

    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(duration)
    meshcat.Flush()
    visualizer.PublishRecording()
    

    return simulator, plant

In [14]:
#meshcat.Delete()

# Get string describing scenario
scenario_data = get_library_scenario_data()

#Setting up all drake simulation objects:
builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)

# Allowing the Parser to access all of the aspects in the environment including the robot (iiwa)
parser.AddModelsFromUrl("file:///workspaces/RobotLibrarian/hwstation/objects/mobile_iiwa.dmd.yaml")


[ModelInstanceIndex(2), ModelInstanceIndex(3)]

In [15]:
def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    # AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )
    # visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)


    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("mobile_iiwa.desired_state")
    )

    diagram = builder.Build()
    plant.Finalize()

    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(duration)
    meshcat.Flush()
    visualizer.PublishRecording()
    

    return simulator, plant

In [16]:
inputPort = station.GetInputPort("mobile_iiwa.desired_state")
# x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(sim_context)
# x0

In [17]:
from hwstation.utils import visualize_diagram

In [18]:
simulator, station_plant = BuildAndSimulateTrajectory(q_traj, traj_wsg_command, 11.0)

: 

## CODE BELOW THIS POINT IS EXPERIMENTAL

In [61]:
# from hwstation.utils import visualize_diagram
# visualize_diagram(diagram)

In [51]:
# builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)
# parser.AddModelsFromUrl("file:///workspaces/RobotLibrarian/hwstation/objects/mobile_iiwa.dmd.yaml")



RuntimeError: DiagramBuilder::Connect: Mismatched vector sizes while connecting output port y0 of System drake/systems/TrajectorySource@00005555761b8150 (size 12) to input port mobile_iiwa.desired_state of System station (size 20)

In [ ]:
y_bin_grasp_selector = builder.AddSystem(
    GraspSelector(
        plant,
        plant.GetModelInstanceByName("bin0"),
        camera_body_indices=[
            plant.GetBodyIndices(plant.GetModelInstanceByName("camera0"))[
                0
            ],
            plant.GetBodyIndices(plant.GetModelInstanceByName("camera1"))[
                0
            ],
            plant.GetBodyIndices(plant.GetModelInstanceByName("camera2"))[
                0
            ],
        ],
    )
)
builder.Connect(
    to_point_cloud["camera0"].get_output_port(),
    y_bin_grasp_selector.get_input_port(0),
)
builder.Connect(
    to_point_cloud["camera1"].get_output_port(),
    y_bin_grasp_selector.get_input_port(1),
)
builder.Connect(
    to_point_cloud["camera2"].get_output_port(),
    y_bin_grasp_selector.get_input_port(2),
)
builder.Connect(
    station.GetOutputPort("body_poses"),
    y_bin_grasp_selector.GetInputPort("body_poses"),
)

x_bin_grasp_selector = builder.AddSystem(
    GraspSelector(
        plant,
        plant.GetModelInstanceByName("bin1"),
        camera_body_indices=[
            plant.GetBodyIndices(plant.GetModelInstanceByName("camera3"))[
                0
            ],
            plant.GetBodyIndices(plant.GetModelInstanceByName("camera4"))[
                0
            ],
            plant.GetBodyIndices(plant.GetModelInstanceByName("camera5"))[
                0
            ],
        ],
    )
)
builder.Connect(
    to_point_cloud["camera3"].get_output_port(),
    x_bin_grasp_selector.get_input_port(0),
)
builder.Connect(
    to_point_cloud["camera4"].get_output_port(),
    x_bin_grasp_selector.get_input_port(1),
)
builder.Connect(
    to_point_cloud["camera5"].get_output_port(),
    x_bin_grasp_selector.get_input_port(2),
)
builder.Connect(
    station.GetOutputPort("body_poses"),
    x_bin_grasp_selector.GetInputPort("body_poses"),
)

planner = builder.AddSystem(Planner(plant))
builder.Connect(
    station.GetOutputPort("body_poses"), planner.GetInputPort("body_poses")
)
builder.Connect(
    x_bin_grasp_selector.get_output_port(),
    planner.GetInputPort("x_bin_grasp"),
)
builder.Connect(
    y_bin_grasp_selector.get_output_port(),
    planner.GetInputPort("y_bin_grasp"),
)
builder.Connect(
    station.GetOutputPort("wsg.state_measured"),
    planner.GetInputPort("wsg_state"),
)
builder.Connect(
    station.GetOutputPort("iiwa.position_measured"),
    planner.GetInputPort("iiwa_position"),
)

robot = station.GetSubsystemByName(
    "iiwa.controller"
).get_multibody_plant_for_control()

# Set up differential inverse kinematics.
diff_ik = AddIiwaDifferentialIK(builder, robot)
builder.Connect(planner.GetOutputPort("X_WG"), diff_ik.get_input_port(0))
builder.Connect(
    station.GetOutputPort("iiwa.state_estimated"),
    diff_ik.GetInputPort("robot_state"),
)
builder.Connect(
    planner.GetOutputPort("reset_diff_ik"),
    diff_ik.GetInputPort("use_robot_state"),
)

builder.Connect(
    planner.GetOutputPort("wsg_position"),
    station.GetInputPort("wsg.position"),
)

# The DiffIK and the direct position-control modes go through a PortSwitch
switch = builder.AddSystem(PortSwitch(7))
builder.Connect(
    diff_ik.get_output_port(), switch.DeclareInputPort("diff_ik")
)
builder.Connect(
    planner.GetOutputPort("iiwa_position_command"),
    switch.DeclareInputPort("position"),
)
builder.Connect(
    switch.get_output_port(), station.GetInputPort("iiwa.position")
)
builder.Connect(
    planner.GetOutputPort("control_mode"),
    switch.get_port_selector_input_port(),
)

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, station.GetOutputPort("query_object"), meshcat
)
diagram = builder.Build()

In [180]:
def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    builder = DiagramBuilder()
    print("MakeManipulationStation")
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="file:///workspaces/RobotLibrarian/hwstation/objects/library_setup.dmd.yaml",
            time_step=1e-3,
            iiwa_prefix="mobile_iiwa",
            wsg_prefix="wsg",
        )
    )
    print("Get plant")
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    print("Adding systems")
    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    print("Visualizer")
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    print("Builder connect")
    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
    )
    builder.Connect(
        g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
    )
    print("builder.Build()")
    diagram = builder.Build()


In [181]:
# def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
#     """Simulate trajectory for manipulation station.
#     @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
#     @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
#     """
#     AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

#     q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
#     g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

#     visualizer = MeshcatVisualizer.AddToBuilder(
#         builder,
#         station.GetOutputPort("query_object"),
#         meshcat,
#         MeshcatVisualizerParams(delete_on_initialization_event=False),
#     )

#     builder.Connect(
#         q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
#     )
#     builder.Connect(
#         g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
#     )

#     diagram = builder.Build()

#     simulator = Simulator(diagram)
#     visualizer.StartRecording(False)
#     simulator.AdvanceTo(duration)
#     visualizer.PublishRecording()

#     return simulator, plant

In [182]:
q_knots = np.array(q_knots)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, :10].T)
simulator, station_plant = BuildAndSimulateTrajectory(q_traj, traj_wsg_command, 11.0)

MakeManipulationStation


SystemExit: Failure at systems/controllers/inverse_dynamics_controller.cc:41 in SetUp(): condition 'num_positions == dim' failed.

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
base_trans = base_y_pose.translation()

array([1. , 1.5, 0. ])

In [ ]:
def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="package://manipulation/manipulation_station_with_cupboard.dmd.yaml",
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
    )
    builder.Connect(
        g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(duration)
    visualizer.PublishRecording()

    return simulator, plant

In [41]:
class PlannerState(Enum):
    WAIT_FOR_OBJECTS_TO_SETTLE = 1
    PICKING_BOOK_FROM_TABLE = 2
    GOING_TO_SHELF = 3
    PLACING_BOOK_ON_SHELF = 4
    GO_BACK_TO_TABLE = 5

In [ ]:
class Planner(LeafSystem):
    
    """A Planner system.

    Input Ports:
    - body_poses: RigidTransforms given the desired frame of the gripper
    - table_book_grasp: (cost,RigidTransform) giving the desired frame to grasp a book on the table
    - shelf_book_grasp: (cost, RididTransform) giving the desired frame to put a book on the shelf
    - wsg_state: a vector of size 2 giving the WSG commands (I supose each fingers position)
    - iiwa_joints: a vector of size 7 for joint positions
    - iiwa_base: a vector of size 2 for x- and y- coordinates of base

    States:
    - self._mode_index: current mode of the planner
    - self._traj_X_G_index: trajectory of gripper frame
    - self._traj_wsg_index: command trajectory of WSG fingers?
    - self._times_index: dictionary of times
    - self._attempts_index: attempts of grasping something

    Output ports:
    - 
    -
    -
    """

    def __init__(self, plant):
        LeafSystem.__init__(self)

        self._gripper_body_index = plant.GetBodyByName("body").index()
        
        #********
        #Declare input ports
        #********
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )
        self._table_book_grasp_index = self.DeclareAbstractInputPort(
            "table_book_grasp", AbstractValue.Make((np.inf, RigidTransform()))
        ).get_index()
        self._shelf_book_grasp_index = self.DeclareAbstractInputPort(
            "shelf_book_grasp", AbstractValue.Make((np.inf, RigidTransform()))
        ).get_index()

        #Declares a vector-valued input port of size 2:
        self._wsg_state_index = self.DeclareVectorInputPort(
            "wsg_state", 2
        ).get_index()

        # For GoHome mode.
        num_iiwa_joints = 7
        self._iiwa_joint_position_index = self.DeclareVectorInputPort(
            "iiwa_joints", num_iiwa_joints
        ).get_index()

        num_iiwa_base_coords = 2
        self._iiwa_base_coords_index = self.DeclareVectorInputPort(
            "iiwa_pos", num_iiwa_base_coords
        ).get_index()

        #********
        #Declare states of the system
        #********

        #Abstract state giving the mode:
        #A Context obtained with CreateDefaultContext() will contain this abstract state 
        #variable initially set to a clone of the model_value given here. The actual concrete 
        # type is always preserved.
        self._mode_index = self.DeclareAbstractState(
            AbstractValue.Make(PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE)
        )
        self._traj_X_G_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose())
        )
        self._traj_wsg_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )
        self._times_index = self.DeclareAbstractState(
            AbstractValue.Make({"initial": 0.0})
        )
        #Declares a discrete state group with num_state_variables state variables, 
        #stored in a BasicVector initialized to be all-zero.
        self._attempts_index = self.DeclareDiscreteState(1)

        #********
        #Declare output ports of the system
        #********
        self.DeclareAbstractOutputPort(
            "X_WG",
            lambda: AbstractValue.Make(RigidTransform()),
            self.CalcGripperPose)

        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)

        self.DeclareAbstractOutputPort(
            "control_mode",
            lambda: AbstractValue.Make(InputPortIndex(0)),
            self.CalcControlMode,
        )

        self.DeclareAbstractOutputPort(
            "reset_diff_ik",
            lambda: AbstractValue.Make(False),
            self.CalcDiffIKReset,
        )

        self._q0_index = self.DeclareDiscreteState(num_positions)  # for q0
        self._traj_q_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )
        #DEBUG: MERGE TWO STATES?
        self.DeclareVectorOutputPort(
            "iiwa_base_command", num_iiwa_base, self.CalcIiwaBase
        )
        self.DeclareVectorOutputPort(
            "iiwa_joint_command", num_iiwa_joints, self.CalcIiwaJoint
        )

        #********
        #Declare initialization
        #********
        self.DeclareInitializationDiscreteUpdateEvent(self.Initialize)

        #********
        #Declare periodic updates
        #********
        # Declares that an UnrestrictedUpdate event should occur periodically and 
        # that it should invoke the Update method every 0.1 second at 0.0 offset
        self.DeclarePeriodicUnrestrictedUpdateEvent(0.1, 0.0, self.Update)

    def Update(self, context, state):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        current_time = context.get_time()
        times = context.get_abstract_state(int(self._times_index)).get_value()

        #Wait for objects to settle:
        if mode == PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE:
            if context.get_time() - times["initial"] > 1.0:
                self.Plan(context, state)
            return
        
        elif mode == PlannerState.GO_HOME:
            sys.exit("GOING HOME IS NOT IMPLEMENTED YET.")
            # traj_q = context.get_mutable_abstract_state(
            #     int(self._traj_q_index)
            # ).get_value()
            # if not traj_q.is_time_in_range(current_time):
            #     self.Plan(context, state)
            return

        # If we are between pick and place and the gripper is closed, then
        # we've missed or dropped the object.  Time to replan.
        if (
            current_time > times["postpick"]
            and current_time < times["preplace"]
        ):
            wsg_state = self.get_input_port(self._wsg_state_index).Eval(
                context
            )
            if wsg_state[0] < 0.01:
                attempts = state.get_mutable_discrete_state(
                    int(self._attempts_index)
                ).get_mutable_value()
                if attempts[0] > 5:
                    # If I've failed 5 times in a row, then switch bins.
                    print(
                        "Switching to the other bin after 5 consecutive failed attempts"
                    )
                    attempts[0] = 0
                    if mode == PlannerState.PICKING_FROM_X_BIN:
                        state.get_mutable_abstract_state(
                            int(self._mode_index)
                        ).set_value(PlannerState.PICKING_FROM_Y_BIN)
                    else:
                        state.get_mutable_abstract_state(
                            int(self._mode_index)
                        ).set_value(PlannerState.PICKING_FROM_X_BIN)
                    self.Plan(context, state)
                    return

                attempts[0] += 1
                state.get_mutable_abstract_state(
                    int(self._mode_index)
                ).set_value(PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE)
                times = {"initial": current_time}
                state.get_mutable_abstract_state(
                    int(self._times_index)
                ).set_value(times)
                X_G = self.get_input_port(0).Eval(context)[
                    int(self._gripper_body_index)
                ]
                state.get_mutable_abstract_state(
                    int(self._traj_X_G_index)
                ).set_value(
                    PiecewisePose.MakeLinear(
                        [current_time, np.inf], [X_G, X_G]
                    )
                )
                return

        traj_X_G = context.get_abstract_state(
            int(self._traj_X_G_index)
        ).get_value()
        if not traj_X_G.is_time_in_range(current_time):
            self.Plan(context, state)
            return

        X_G = self.get_input_port(0).Eval(context)[
            int(self._gripper_body_index)
        ]
        # if current_time > 10 and current_time < 12:
        #    self.GoHome(context, state)
        #    return
        if (
            np.linalg.norm(
                traj_X_G.GetPose(current_time).translation()
                - X_G.translation()
            )
            > 0.2
        ):
            # If my trajectory tracking has gone this wrong, then I'd better
            # stop and replan.  TODO(russt): Go home, in joint coordinates,
            # instead.
            self.GoHome(context, state)
            return

    def GoToHomePosition(self, context, state):
        print("Replanning due to large tracking error.")
        state.get_mutable_abstract_state(int(self._mode_index)).set_value(
            PlannerState.GO_TO_HOME_POSITION
        )
        q = self.get_input_port(self._iiwa_position_index).Eval(context)
        q0 = copy(context.get_discrete_state(self._q0_index).get_value())
        q0[0] = q[0]  # Safer to not reset the first joint.

        current_time = context.get_time()
        q_traj = PiecewisePolynomial.FirstOrderHold(
            [current_time, current_time + 5.0], np.vstack((q, q0)).T
        )
        state.get_mutable_abstract_state(int(self._traj_q_index)).set_value(
            q_traj
        )

    def Plan(self, context, state):
        mode = copy(
            state.get_mutable_abstract_state(int(self._mode_index)).get_value()
        )

        X_G = {
            "initial": self.get_input_port(0).Eval(context)[
                int(self._gripper_body_index)
            ]
        }

        cost = np.inf
        for i in range(5):
            if mode == PlannerState.PICKING_FROM_Y_BIN:
                cost, X_G["pick"] = self.get_input_port(
                    self._y_bin_grasp_index
                ).Eval(context)
                if np.isinf(cost):
                    mode = PlannerState.PICKING_FROM_X_BIN
            else:
                cost, X_G["pick"] = self.get_input_port(
                    self._x_bin_grasp_index
                ).Eval(context)
                if np.isinf(cost):
                    mode = PlannerState.PICKING_FROM_Y_BIN
                else:
                    mode = PlannerState.PICKING_FROM_X_BIN

            if not np.isinf(cost):
                break

        assert not np.isinf(
            cost
        ), "Could not find a valid grasp in either bin after 5 attempts"
        state.get_mutable_abstract_state(int(self._mode_index)).set_value(mode)

        # TODO(russt): The randomness should come in through a random input
        # port.
        if mode == PlannerState.PICKING_FROM_X_BIN:
            # Place in Y bin:
            X_G["place"] = RigidTransform(
                RollPitchYaw(-np.pi / 2, 0, 0),
                [rng.uniform(-0.25, 0.15), rng.uniform(-0.6, -0.4), 0.3],
            )
        else:
            # Place in X bin:
            X_G["place"] = RigidTransform(
                RollPitchYaw(-np.pi / 2, 0, np.pi / 2),
                [rng.uniform(0.35, 0.65), rng.uniform(-0.12, 0.28), 0.3],
            )

        X_G, times = MakeGripperFrames(X_G, t0=context.get_time())
        print(
            f"Planned {times['postplace'] - times['initial']} second trajectory in mode {mode} at time {context.get_time()}."
        )
        state.get_mutable_abstract_state(int(self._times_index)).set_value(
            times
        )

        if False:  # Useful for debugging
            AddMeshcatTriad(meshcat, "X_Oinitial", X_PT=X_O["initial"])
            AddMeshcatTriad(meshcat, "X_Gprepick", X_PT=X_G["prepick"])
            AddMeshcatTriad(meshcat, "X_Gpick", X_PT=X_G["pick"])
            AddMeshcatTriad(meshcat, "X_Gplace", X_PT=X_G["place"])

        traj_X_G = MakeGripperPoseTrajectory(X_G, times)
        traj_wsg_command = MakeGripperCommandTrajectory(times)

        state.get_mutable_abstract_state(int(self._traj_X_G_index)).set_value(
            traj_X_G
        )
        state.get_mutable_abstract_state(int(self._traj_wsg_index)).set_value(
            traj_wsg_command
        )

    def start_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index))
            .get_value()
            .start_time()
        )

    def end_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index))
            .get_value()
            .end_time()
        )

    def CalcGripperPose(self, context, output):
        context.get_abstract_state(int(self._mode_index)).get_value()

        traj_X_G = context.get_abstract_state(
            int(self._traj_X_G_index)
        ).get_value()
        if traj_X_G.get_number_of_segments() > 0 and traj_X_G.is_time_in_range(
            context.get_time()
        ):
            # Evaluate the trajectory at the current time, and write it to the
            # output port.
            output.set_value(
                context.get_abstract_state(int(self._traj_X_G_index))
                .get_value()
                .GetPose(context.get_time())
            )
            return

        # Command the current position (note: this is not particularly good if the velocity is non-zero)
        output.set_value(
            self.get_input_port(0).Eval(context)[int(self._gripper_body_index)]
        )

    def CalcWsgPosition(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        opened = np.array([0.107])
        np.array([0.0])

        if mode == PlannerState.GO_HOME:
            # Command the open position
            output.SetFromVector([opened])
            return

        traj_wsg = context.get_abstract_state(
            int(self._traj_wsg_index)
        ).get_value()
        if traj_wsg.get_number_of_segments() > 0 and traj_wsg.is_time_in_range(
            context.get_time()
        ):
            # Evaluate the trajectory at the current time, and write it to the
            # output port.
            output.SetFromVector(traj_wsg.value(context.get_time()))
            return

        # Command the open position
        output.SetFromVector([opened])

    def CalcControlMode(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        if mode == PlannerState.GO_HOME:
            output.set_value(InputPortIndex(2))  # Go Home
        else:
            output.set_value(InputPortIndex(1))  # Diff IK

    def CalcDiffIKReset(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        if mode == PlannerState.GO_HOME:
            output.set_value(True)
        else:
            output.set_value(False)

    def Initialize(self, context, discrete_state):
        discrete_state.set_value(
            int(self._q0_index),
            self.get_input_port(int(self._iiwa_position_index)).Eval(context),
        )

    def CalcIiwaBase(self, context, output):
        pass #Not implemented yet
        # traj_q = context.get_mutable_abstract_state(
        #     int(self._traj_q_index)
        # ).get_value()

        # output.SetFromVector(traj_q.value(context.get_time()))

    def CalcIiwaJoint(self, context, output):
        traj_q = context.get_mutable_abstract_state(
            int(self._traj_q_index)
        ).get_value()

        output.SetFromVector(traj_q.value(context.get_time()))



In [27]:
# def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
#     """Simulate trajectory for manipulation station.
#     @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
#     @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
#     """
#     builder = DiagramBuilder()
#     station = builder.AddSystem(
#         MakeManipulationStation(
#             filename="package://manipulation/manipulation_station_with_cupboard.dmd.yaml",
#             time_step=1e-3,
#         )
#     )
#     plant = station.GetSubsystemByName("plant")
#     scene_graph = station.GetSubsystemByName("scene_graph")
#     AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

duration = 1.0
q_traj_system = builder.AddSystem(TrajectorySource(trajectory))
g_traj_system = builder.AddSystem(TrajectorySource(traj_wsg_command))

visualizer = MeshcatVisualizer.AddToBuilder(
    environment_builder,
    environment_station.GetOutputPort("query_object"),
    meshcat,
    MeshcatVisualizerParams(delete_on_initialization_event=False),
)

environment_builder.Connect(
    q_traj_system.get_output_port(), environment_station.GetInputPort("iiwa_position")
)
environment_builder.Connect(
    g_traj_system.get_output_port(), environment_station.GetInputPort("wsg_position")
)

#diagram = environment_builder.Build()

simulator = Simulator(environment)
visualizer.StartRecording(False)
simulator.AdvanceTo(duration)
visualizer.PublishRecording()

SystemExit: Failure at systems/primitives/trajectory_source.cc:25 in TrajectorySource(): condition 'trajectory.cols() == 1' failed.

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [54]:
from pydrake.multibody import inverse_kinematics

In [68]:
max_time = max(traj_times.values())
t_lst = np.linspace(0, max_time, 30)
pose_lst = []
for t in t_lst:
    #AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=0.2)
    pose_lst.append(RigidTransform(trajectory.value(t)))

In [94]:
len(ik.q())

19

In [47]:
# def create_q_knots(pose_lst):
#     """Convert end-effector pose list to joint position list using series of
#     InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
#     contain gripper joints, but these should not matter to the constraints.
#     @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
#     @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
#     """
q_knots = []
world_frame = plant.world_frame()
gripper_frame = plant.GetFrameByName("body")
q_nominal = np.array(
    [0.0, 0.0, 0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
)  # nominal joint angles for joint-centering.

def AddOrientationConstraint(ik, R_WG, bounds):
    """Add orientation constraint to the ik problem. Implements an inequality
    constraint where the axis-angle difference between f_R(q) and R_WG must be
    within bounds. Can be translated to:
    ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
    """
    ik.AddOrientationConstraint(
        frameAbar=world_frame,
        R_AbarA=R_WG,
        frameBbar=gripper_frame,
        R_BbarB=RotationMatrix(),
        theta_bound=bounds,
    )

def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
    """Add position constraint to the ik problem. Implements an inequality
    constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
    translated to
    ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
    """
    ik.AddPositionConstraint(
        frameA=world_frame,
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower,
        p_AQ_upper=p_WG_upper,
    )

for i in range(len(pose_lst)):
    ik = inverse_kinematics.InverseKinematics(environment_plant)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram

    #### Modify here ###############################
    X_WG = pose_lst[i]
    p_WG = X_WG.translation()
    R_WG = X_WG.rotation()
    d = 0.05
    p_WG_adjustment = np.array([0.0, 0.0, d / 2.0])
    #AddPositionConstraint(ik, p_WG - p_WG_adjustment, p_WG + p_WG_adjustment)
    #AddOrientationConstraint(ik, R_WG, 0.05)
    prog.AddQuadraticErrorCost(np.identity(len(q_variables)), q_nominal, q_variables)
    if i == 0:
        prog.SetInitialGuess(q_variables, q_nominal)
    else:
        prog.SetInitialGuess(q_variables, q_knots[i-1])
    
    ################################################

    result = Solve(prog)

    assert result.is_success()

    q_knots.append(result.GetSolution(q_variables))



NameError: name 'pose_lst' is not defined